In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Gumbel Softmax


* Pre-print, published in ICLR 2017 https://arxiv.org/pdf/1611.01144.pdf


## Experiments
### 1 Stochastic Gradient Estimation
* **Dataset**: We use the MNIST dataset with fixed binarization for training and evaluation
* **Tricks**: We also found that variance normalization was necessary
* **Network**: We used sigmoid activation functions for binary (Bernoulli) neural networks and softmax activations for categorical variables.
* **Training**: Models were trained using stochastic gradient descent with momentum 0.9.
* **Learning rates**:  are chosen from {3e−5, 1e−5, 3e−4, 1e−4, 3e−3, 1e−3}; we select the best learning rate for each estimator using the MNIST validation set, and report performance on the test set.

### 2) Structured output prediction with stochastic binary networks

### 3) Generative modelling with variational Autoencoders

### 4) Generative semi supervised classification

# Requirements

In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
import pathlib
import os
import matplotlib.pyplot as plt
import numpy as np

np.set_printoptions(precision=4)

In [3]:
import pandas as pd

# Load and preprocess data

In [ ]:
import tensorflow_datasets as tfds
mnist_data = tfds.load("binarized_mnist", data_dir="/tf/2016/gumbel-softmax/data")
mnist_train, mnist_test = mnist_data["train"], mnist_data["test"]
assert isinstance(mnist_train, tf.data.Dataset)

Shuffling and writing examples to /tf/2016/gumbel-softmax/data/binarized_mnist/1.0.0.incompleteWKJG8V/binarized_mnist-train.tfrecord


Shuffling and writing examples to /tf/2016/gumbel-softmax/data/binarized_mnist/1.0.0.incompleteWKJG8V/binarized_mnist-validation.tfrecord
